In this tutorial, we show how to customize your search space with
[AutoModel](/auto_model/#automodel-class) and how to implement your own block as search space.
This API is mainly for advanced users who already know what their model should look like.
## Customized Search Space
First, let us see how we can build the following neural network using the building blocks in AutoKeras.
<div class="mermaid">
graph LR
    id1(ImageInput) --> id2(Normalization)
    id2 --> id3(Image Augmentation)
    id3 --> id4(Convolutional)
    id3 --> id5(ResNet V2)
    id4 --> id6(Merge)
    id5 --> id6
    id6 --> id7(Classification Head)
</div>
We can make use of the [AutoModel](/auto_model/#automodel-class) API in AutoKeras to implemented as follows.
The usage is the same as the [Keras functional API](https://www.tensorflow.org/guide/keras/functional).
Since this is just a demo, we use small amount of `max_trials` and `epochs`.

In [1]:
import autokeras as ak

In [2]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node1 = ak.ConvBlock()(output_node)
output_node2 = ak.ResNetBlock(version='v2')(output_node)
output_node = ak.Merge()([output_node1, output_node2])
output_node = ak.ClassificationHead()(output_node)

auto_model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    overwrite=True,
    max_trials=1)

In [3]:
from tensorflow.keras.datasets import mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape) # (60000, 28, 28)
print(y_train.shape) # (60000,)
print(y_train[:3]) # array([7, 2, 1], dtype=uint8)

# Feed the AutoModel with training data.
auto_model.fit(x_train[:100], y_train[:100], epochs=1)
# Predict with the best model.
predicted_y = auto_model.predict(x_test)
# Evaluate the best model with testing data.
print(auto_model.evaluate(x_test, y_test))

Trial 1 Complete [00h 00m 24s]
val_loss: 2.2638494968414307

Best val_loss So Far: 2.2638494968414307
Total elapsed time: 00h 00m 24s
INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - 9s 2s/step - loss: 2.3706 - accuracy: 0.2400
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets
313/313 [==============================] - 106s 340ms/step - loss: 2.2315 - accuracy: 0.3097
[2.231471538543701, 0.30970001220703125]
